In [4]:
!wget --no-clobber https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip -n data.zip

File ‘data.zip’ already there; not retrieving.

Archive:  data.zip


In [5]:
import numpy as np
import tensorflow as tf

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

2024-12-03 00:01:25.700291: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 00:01:26.044632: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 00:01:26.361939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-03 00:01:26.625441: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-03 00:01:26.705755: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 00:01:27.205417: I tensorflow/core/platform/cpu_feature_gu

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input
from tensorflow.keras.optimizers import SGD

def create_model():
    model = Sequential([
        # Input layer with specified shape
        Input(shape=(200, 200, 3)),
        Conv2D(32, (3, 3), activation='relu'),
        # Max pooling layer
        MaxPooling2D(pool_size=(2, 2)),
        
        # Flatten layer to convert 2D feature maps to 1D vector
        Flatten(),
        
        # Dense hidden layer
        Dense(64, activation='relu'),
        
        # Output layer with sigmoid activation for binary classification
        Dense(1, activation='sigmoid')
    ])
    
    # Compile the model with specified optimizer
    optimizer = SGD(learning_rate=0.002, momentum=0.8)
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create the model
model = create_model()

# Display model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 313632)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    20,072,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,073,473 (76.57 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create image data generator with rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255  # Normalize pixel values to [0,1]
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

# Load and prepare the training data
train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(200, 200),  # Resize images to match our model's input shape
    batch_size=20,
    class_mode='binary',     # For binary classification
    shuffle=True
)

# Load and prepare the test data
test_generator = test_datagen.flow_from_directory(
    'data/test',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=False
)

# Print class mapping to understand label encoding
print("Class mapping:", train_generator.class_indices)

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.
Class mapping: {'curly': 0, 'straight': 1}


In [8]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

/root/miniconda3/envs/ml-zoomcamp-ubuntu/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.4992 - loss: 0.7309 - val_accuracy: 0.5423 - val_loss: 0.6831
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 38s 967ms/step - accuracy: 0.6333 - loss: 0.6761 - val_accuracy: 0.5672 - val_loss: 0.6716
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 38s 936ms/step - accuracy: 0.6396 - loss: 0.6506 - val_accuracy: 0.6219 - val_loss: 0.6371
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 38s 935ms/step - accuracy: 0.6653 - loss: 0.6085 - val_accuracy: 0.6219 - val_loss: 0.6305
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 38s 961ms/step - accuracy: 0.6583 - loss: 0.6091 - val_accuracy: 0.5821 - val_loss: 0.6665
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 35s 875ms/step - accuracy: 0.6802 - loss: 0.5923 - val_accuracy: 0.6169 - val_loss: 0.6290
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 35s 879ms/step - accuracy: 0.6764 - loss: 0.5745 - val_accuracy: 0.6269 - val_loss: 0.6429
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 37s 918ms/step - accuracy: 0.6988 - loss: 0.5423 - val_accurac

In [9]:
# Get training accuracy and loss values
train_accuracy = history.history['accuracy']
train_loss = history.history['loss']

# Calculate median accuracy
median_accuracy = np.median(train_accuracy)
print(f"Median training accuracy: {median_accuracy:.3f}")

# Calculate standard deviation of loss
std_loss = np.std(train_loss)
print(f"Standard deviation of training loss: {std_loss:.3f}")

Median training accuracy: 0.678
Standard deviation of training loss: 0.057


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Training data generator with augmentations
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load and prepare the training data
train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Found 800 images belonging to 2 classes.
Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.6260 - loss: 0.6602 - val_accuracy: 0.6468 - val_loss: 0.6530
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.6571 - loss: 0.6129 - val_accuracy: 0.6716 - val_loss: 0.5809
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.6716 - loss: 0.6104 - val_accuracy: 0.6766 - val_loss: 0.5995
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.6647 - loss: 0.6072 - val_accuracy: 0.6667 - val_loss: 0.5818
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.6764 - loss: 0.6102 - val_accuracy: 0.6766 - val_loss: 0.5857
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.6748 - loss: 0.5936 - val_accuracy: 0.6816 - val_loss: 0.5720
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.6793 - loss: 0.5846 - val_accuracy: 0.6915 - val_loss: 0.6087
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.6956 - loss: 

In [11]:
# Get test/validation loss values
test_loss = history.history['val_loss']

# Calculate mean test loss
mean_test_loss = np.mean(test_loss)
print(f"Mean test loss: {mean_test_loss:.3f}")

# Print all test loss values for verification
print("\nTest loss values per epoch:", test_loss)

Mean test loss: 0.595

Test loss values per epoch: [0.6529668569564819, 0.5808520317077637, 0.5994570255279541, 0.5817888379096985, 0.5857492089271545, 0.571992039680481, 0.6087288856506348, 0.5932213068008423, 0.597454309463501, 0.5801623463630676]


In [12]:
# Get test/validation accuracy values
test_accuracy = history.history['val_accuracy']

# Calculate mean of last 5 epochs (indices 5 to 9, which are epochs 6 to 10)
last_5_accuracy = test_accuracy[5:]
mean_last_5_accuracy = np.mean(last_5_accuracy)

print(f"Mean test accuracy for last 5 epochs: {mean_last_5_accuracy:.3f}")
print("\nTest accuracy values for last 5 epochs:", last_5_accuracy)

Mean test accuracy for last 5 epochs: 0.688

Test accuracy values for last 5 epochs: [0.6815920472145081, 0.6915422677993774, 0.7014925479888916, 0.6716417670249939, 0.6915422677993774]
